In [2]:
import numpy as np 
import pandas as pd

In [3]:
data  = pd.read_csv('../static/capes.csv')
data.head()

,Unnamed: 0,Instructor,Course,Quarter,TotalEnrolledinCourse,Total CAPEsGiven,PercentageRecommendedClass,PercentageRecommendedProfessor,StudyHoursperWeek,AverageGradeExpected,AverageGradeReceived,EvalulationURL,GradedCourse,AverageGradeExpected1,AverageGradeReceived1,CourseID,CourseName,CourseSection
0,0,Rampure Suraj,DSC 10 - Principles of Data Science (A),SP23,117,92,97.8,98.9,8.32,3.55,3.21,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,1,B+,B,DSC 10,Principles of Data Science,A
1,1,Rampure Suraj,DSC 10 - Principles of Data Science (B),SP23,117,80,82.3,96.2,8.78,3.42,3.14,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,1,B+,B,DSC 10,Principles of Data Science,B
2,2,Salimi Babak,DSC 100 - Intro to Data Management (A),SP23,99,81,87.7,80.0,5.33,3.86,3.77,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,1,A-,A-,DSC 100,Intro to Data Management,A
3,3,Albuyeh Rod,DSC 102 - Systems for Scalable Analytics (A),SP23,112,77,93.5,92.2,6.74,3.73,3.53,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,1,A-,B+,DSC 102,Systems for Scalable Analytics,A
4,4,Gupta Amarnath,DSC 104 - Beyond Relational Data Mgmt (A),SP23,18,6,100.0,80.0,6.10,4.00,NaN,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,1,A,NaN,DSC 104,Beyond Relational Data Mgmt,A


In [4]:
data['StudyHoursperWeek'].min(), data['StudyHoursperWeek'].max()

(0.65, 18.83)

In [5]:
data['AverageGradeExpected'].min(), data['AverageGradeExpected'].max()

(2.43, 4.0)

In [6]:
def find_range(col):  
    return data[col].min(), data[col].max() 

print(find_range("AverageGradeExpected"))
print(find_range('AverageGradeReceived'))

(2.43, 4.0)
(2.47, 4.0)


In [7]:
data['StudyHoursperWeek']

0      8.32
1      8.78
2      5.33
3      6.74
4      6.10
       ... 
263    7.27
264    9.89
265    8.18
266    6.63
267    8.50
Name: StudyHoursperWeek, Length: 268, dtype: float64

In [8]:
data[data['AverageGradeExpected'] == 2.43]

,Unnamed: 0,Instructor,Course,Quarter,TotalEnrolledinCourse,Total CAPEsGiven,PercentageRecommendedClass,PercentageRecommendedProfessor,StudyHoursperWeek,AverageGradeExpected,AverageGradeReceived,EvalulationURL,GradedCourse,AverageGradeExpected1,AverageGradeReceived1,CourseID,CourseName,CourseSection
89,89,Dumitriu Ioana,DSC 155 - Hidden Data in Random Matrices (A),WI22,18,8,57.1,100.0,10.75,2.43,NaN,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,1,C+,NaN,DSC 155,Hidden Data in Random Matrices,A


In [9]:
grade_mapping = {
    'A+': 4.0,
    'A': 4.0,
    'A-': 3.7,
    'B+': 3.3,
    'B': 3.0,
    'B-': 2.7,
    'C+': 2.3,
    'C': 2.0,
    'C-': 1.7
}

In [10]:
data['AverageGradeExpected']

0      3.55
1      3.42
2      3.86
3      3.73
4      4.00
       ... 
263    3.59
264    3.43
265    3.29
266    3.44
267    3.29
Name: AverageGradeExpected, Length: 268, dtype: float64

In [11]:
data["Course"].shape

(268,)

In [23]:
data['Course'].str.split(' - ')

0           [DSC 10, Principles of Data Science (A)]
1           [DSC 10, Principles of Data Science (B)]
2            [DSC 100, Intro to Data Management (A)]
3      [DSC 102, Systems for Scalable Analytics (A)]
4         [DSC 104, Beyond Relational Data Mgmt (A)]
                           ...                      
263        [DSC 10, Introduction to Programming (D)]
264           [DSC 20, Intro to Data Structures (B)]
265        [DSC 10, Introduction to Programming (A)]
266        [DSC 10, Introduction to Programming (B)]
267        [DSC 10, Introduction to Programming (C)]
Name: Course, Length: 268, dtype: object

In [ ]:
"